In [ ]:
from datetime import datetime, timedelta
from google.colab import drive
import geopandas as gpd
import pandas as pd
import calendar
import geemap
import ee

In [ ]:
# Authenticated Earth Engine
ee.Authenticate()
ee.Initialize(project='try-spasial')

# ARD Sentinel-1 and Eksport dasarian to assets gee

In [ ]:
# Inisialisasi maps
Map = geemap.Map(basemap='SATELLITE')

# ============================================================================
# DEFINISIAREA OF INTEREST (AOI)
# ============================================================================

# AOI adalah Sawah LBS Kab. Indramayu
AOI_raster = ee.Image('projects/ee-bayuardianto104/assets/Indramayu_Research/LBS_Kab_Indramayu')
AOI_to_vector = AOI_raster.updateMask(AOI_raster.lte(1))
AOI = AOI_to_vector.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=int(1e8)
).filter(ee.Filter.gt('count', 10))

Map.addLayer(AOI, {'color': 'red'}, 'AOI', False)
Map.centerObject(AOI.first().geometry())

# ============================================================================
# FUNGSI KOREKSI CITRA
# ============================================================================

def rem_bdr_noise(img):
    """Remove Border Noise citra - input data dB"""
    edge = img.lt(-35)  # Threshold dalam dB
    properties = img.propertyNames()
    maskedImage = img.mask().And(edge.Not())
    return img.updateMask(maskedImage).copyProperties(img, properties)

def inc_angle(img):
    """Koreksi insiden angle/Terrain correction (flattening) - input data dB"""
    gamma0 = img.expression(
        'i - 10 * log10(cos(angle * pi / 180))', {
        'i': img.select(['VV', 'VH']),
        'angle': img.select('angle'),
        'pi': 3.14159265359
    }).toFloat()
    return img.addBands(gamma0, None, True)

def kor_desp(img):
    """Koreksi geometrik despeckle dengan Median Filter - input data dB"""
    properties = img.propertyNames()
    return img.focalMedian(5).copyProperties(img, properties)

def cal_indices(img):
    """Hitung indeks turunan setelah konversi dB ke linier"""
    # Konversi dari dB ke linear
    VV = img.select('VV').toFloat()
    VH = img.select('VH').toFloat()

    vv_int = VV.expression('10**(vv / 10)', {'vv': VV}).rename('VV_int').toFloat()
    vh_int = VH.expression('10**(vh / 10)', {'vh': VH}).rename('VH_int').toFloat()

    # Hitung indeks menggunakan data linear
    RPI = vh_int.divide(vv_int).rename('RPI').toFloat()
    API = vv_int.add(vh_int).divide(2).rename('API').toFloat()
    NDPI = vv_int.subtract(vh_int).divide(vv_int.add(vh_int)).rename('NDPI').toFloat()
    RVI = vh_int.multiply(4).divide(vv_int.add(vh_int)).rename('RVI').toFloat()

    # Kembalikan semua band dalam linear
    return ee.Image.cat([
        vv_int,
        vh_int,
        img.select('angle'),
        RPI, API, NDPI, RVI
    ]).copyProperties(img, img.propertyNames())

# ============================================================================
# PREPROCESSING SENTINEL-1
# ============================================================================

def S1_preprocessing(param):
    """Preprocessing Sentinel-1 data"""
    s1_col = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filterDate(param['start'], param['end']) \
        .filterBounds(AOI.first().geometry())

    # Apply image correction untuk ARD
    s1_col = s1_col \
        .map(rem_bdr_noise) \
        .map(inc_angle) \
        .map(kor_desp) \
        .map(cal_indices)

    return s1_col

# ============================================================================
# PEMROSESAN DASARIAN
# ============================================================================

def get_last_day_of_month(year, month):
    """Get last day of month"""
    return calendar.monthrange(year, month)[1]

def create_dasarian_periods(year, month):
    """Create dasarian periods for a given year and month"""
    month_str = f"{month:02d}"
    last_day = get_last_day_of_month(year, month)

    periods = [
        {
            'start': f"{year}-{month_str}-01T00:01",
            'end': f"{year}-{month_str}-10T23:59",
            'label': f"{year}-{month_str} Dasarian-1"
        },
        {
            'start': f"{year}-{month_str}-11T00:01",
            'end': f"{year}-{month_str}-20T23:59",
            'label': f"{year}-{month_str} Dasarian-2"
        },
        {
            'start': f"{year}-{month_str}-21T00:01",
            'end': f"{year}-{month_str}-{last_day}T23:59",
            'label': f"{year}-{month_str} Dasarian-3"
        }
    ]

    return periods

def process_dasarian_data(years=[2024], months=list(range(1, 13))):
    """Process dasarian data for given years and months"""
    dasarian_collection = []
    dasarian_labels = []
    export_tasks = []

    for year in years:
        for month in months:
            periods = create_dasarian_periods(year, month)

            for period in periods:
                print(f"Processing: {period['label']}")

                # Proses Sentinel-1 untuk periode dasarian
                processed_image = S1_preprocessing({
                    'start': period['start'],
                    'end': period['end']
                })

                # Hitung jumlah citra dalam periode dasarian
                image_count = processed_image.size()

                # Konversi band ke unsigned int16
                mosaic = processed_image.median()
                scaled_image = mosaic.select(["RPI", "VV_int", "VH_int", "API", "NDPI", "RVI"]) \
                                    .multiply(1000).toUint16() \
                                    .addBands(mosaic.select("angle").multiply(100).toUint16())

                final_image = scaled_image.clip(AOI).set({
                    'system:time_start': ee.Date(period['start']).millis(),
                    'system:time_end': ee.Date(period['end']).advance(1, 'day').millis(),
                    'dasarian': period['label'],
                    'year': year,
                    'month': month,
                    'image_count': image_count,
                    'scale_VV_int': 1000,
                    'scale_VH_int': 1000,
                    'scale_angle': 100
                })

                dasarian_collection.append(final_image)
                dasarian_labels.append(period['label'])

                # Prepare export task
                export_task = {
                    'image': final_image,
                    'description': f'Sentinel1_{period["label"].replace(" ", "_")}',
                    'assetId': f'projects/try-spasial/assets/sentinel1/{period["label"].replace(" ", "_")}',
                    'scale': 30,
                    'region': AOI.geometry(),
                    'maxPixels': int(1e13)
                }

                export_tasks.append(export_task)

    # Gabungkan semua mosaic dasarian menjadi koleksi akhir
    final_dasarian_collection = ee.ImageCollection(dasarian_collection)

    return final_dasarian_collection, dasarian_labels, export_tasks

# ============================================================================
# JALANKAN PEMROSESAN
# ============================================================================

print("Memulai pemrosesan data Sentinel-1...")
final_collection, labels, tasks = process_dasarian_data([2024], list(range(1, 13)))

print(f"Berhasil membuat {len(labels)} periode dasarian")
print("Label periode yang tersedia:")
for i, label in enumerate(labels):
    print(f"{i+1}. {label}")

# ============================================================================
# VISUALISASI DATA
# ============================================================================

def visualize_dasarian(collection, labels, selected_index=0):
    """Visualize selected dasarian period"""
    if selected_index >= len(labels):
        print(f"Index {selected_index} tidak valid. Maksimal index: {len(labels)-1}")
        return

    selected_label = labels[selected_index]
    selected_image = collection.filter(ee.Filter.eq('dasarian', selected_label)).first()

    # Parameter visualisasi
    vis_params = {
        'bands': ['RPI', 'VV_int', 'VH_int'],
        'min': [43, 2, 165],  # Disesuaikan dari nilai asli yang dikali 1000
        'max': [273, 64, 813],  # Disesuaikan dari nilai asli yang dikali 1000
        'gamma': 1.0
    }
    Map.centerObject(AOI, 10)

    Map.addLayer(selected_image, vis_params, selected_label)
    print(f"Menampilkan: {selected_label}")

# visualisasi periode pertama
visualize_dasarian(final_collection, labels, 0)

# ============================================================================
# FUNGSI EXPORT (OPSIONAL)
# ============================================================================

def export_to_asset(export_tasks, start_index=0, end_index=None):
    """Export images to Google Earth Engine Assets"""
    if end_index is None:
        end_index = len(export_tasks)

    print(f"Memulai export dari index {start_index} sampai {end_index-1}...")

    for i in range(start_index, min(end_index, len(export_tasks))):
        task = export_tasks[i]

        export_task = ee.batch.Export.image.toAsset(**task)
        export_task.start()

        print(f"Export task dimulai: {task['description']}")
        print(f"Asset ID: {task['assetId']}")

# ============================================================================
# FUNGSI INTERAKTIF UNTUK MEMILIH PERIODE
# ============================================================================

def create_interactive_selector():
    """Create interactive selector for dasarian periods"""
    import ipywidgets as widgets
    from IPython.display import display

    # Dropdown widget untuk memilih periode
    period_dropdown = widgets.Dropdown(
        options=[(label, i) for i, label in enumerate(labels)],
        value=0,
        description='Periode:',
        style={'description_width': 'initial'}
    )

    def on_period_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            Map.clear_layers()
            Map.addLayer(AOI, {'color': 'red'}, 'AOI', False)
            visualize_dasarian(final_collection, labels, change['new'])

    period_dropdown.observe(on_period_change)

    display(period_dropdown)
    return period_dropdown

# Menyimpan ke asset gee
def export_to_asset(export_tasks, start_index=0, end_index=None):
    """Export images to Google Earth Engine Assets"""
    if end_index is None:
        end_index = len(export_tasks)

    print(f"Memulai export dari index {start_index} sampai {end_index-1}...")

    for i in range(start_index, min(end_index, len(export_tasks))):
        task = export_tasks[i]

        export_task = ee.batch.Export.image.toAsset(**task)
        export_task.start()

        print(f"Export task dimulai: {task['description']}")
        print(f"Asset ID: {task['assetId']}")

export_to_asset(tasks)

# Tampilan Map
Map.addLayer(AOI, {'color': 'red'}, 'AOI', False)
print("Map siap ditampilkan!")
print("Gunakan Map untuk melihat hasil")

# Untuk menggunakan selector dasarian interaktif
# selector = create_interactive_selector()

# Tampilkan Map
Map

# ARD Sentinel-1 and Eksport dasarian to Google Drive

In [ ]:
# Inisialisasi maps
Map = geemap.Map(basemap='SATELLITE')

# ============================================================================
# DEFINISIAREA OF INTEREST (AOI)
# ============================================================================

# AOI adalah Sawah LBS Kab. Indramayu
AOI_raster = ee.Image('projects/ee-bayuardianto104/assets/Indramayu_Research/LBS_Kab_Indramayu')
AOI_to_vector = AOI_raster.updateMask(AOI_raster.lte(1))
AOI = AOI_to_vector.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=int(1e8)
).filter(ee.Filter.gt('count', 10))

Map.addLayer(AOI, {'color': 'red'}, 'AOI', False)
Map.centerObject(AOI.first().geometry())

# ============================================================================
# FUNGSI KOREKSI CITRA
# ============================================================================

def rem_bdr_noise(img):
    """Remove Border Noise citra - input data dB"""
    edge = img.lt(-35)  # Threshold dalam dB
    properties = img.propertyNames()
    maskedImage = img.mask().And(edge.Not())
    return img.updateMask(maskedImage).copyProperties(img, properties)

def inc_angle(img):
    """Koreksi insiden angle/Terrain correction (flattening) - input data dB"""
    gamma0 = img.expression(
        'i - 10 * log10(cos(angle * pi / 180))', {
        'i': img.select(['VV', 'VH']),
        'angle': img.select('angle'),
        'pi': 3.14159265359
    }).toFloat()
    return img.addBands(gamma0, None, True)

def kor_desp(img):
    """Koreksi geometrik despeckle dengan Median Filter - input data dB"""
    properties = img.propertyNames()
    return img.focalMedian(5).copyProperties(img, properties)

def cal_indices(img):
    """Hitung indeks turunan setelah konversi dB ke linier"""
    # Konversi dari dB ke linear
    VV = img.select('VV').toFloat()
    VH = img.select('VH').toFloat()

    vv_int = VV.expression('10**(vv / 10)', {'vv': VV}).rename('VV_int').toFloat()
    vh_int = VH.expression('10**(vh / 10)', {'vh': VH}).rename('VH_int').toFloat()

    # Hitung indeks menggunakan data linear
    RPI = vh_int.divide(vv_int).rename('RPI').toFloat()
    API = vv_int.add(vh_int).divide(2).rename('API').toFloat()
    NDPI = vv_int.subtract(vh_int).divide(vv_int.add(vh_int)).rename('NDPI').toFloat()
    RVI = vh_int.multiply(4).divide(vv_int.add(vh_int)).rename('RVI').toFloat()

    # Kembalikan semua band dalam linear
    return ee.Image.cat([
        vv_int,
        vh_int,
        img.select('angle'),
        RPI, API, NDPI, RVI
    ]).copyProperties(img, img.propertyNames())

# ============================================================================
# PREPROCESSING SENTINEL-1
# ============================================================================

def S1_preprocessing(param):
    """Preprocessing Sentinel-1 data"""
    s1_col = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filterDate(param['start'], param['end']) \
        .filterBounds(AOI.first().geometry())

    # Apply image correction untuk ARD
    s1_col = s1_col \
        .map(rem_bdr_noise) \
        .map(inc_angle) \
        .map(kor_desp) \
        .map(cal_indices)

    return s1_col

# ============================================================================
# PEMROSESAN DASARIAN
# ============================================================================

def get_last_day_of_month(year, month):
    """Get last day of month"""
    return calendar.monthrange(year, month)[1]

def create_dasarian_periods(year, month):
    """Create dasarian periods for a given year and month"""
    month_str = f"{month:02d}"
    last_day = get_last_day_of_month(year, month)

    periods = [
        {
            'start': f"{year}-{month_str}-01T00:01",
            'end': f"{year}-{month_str}-10T23:59",
            'label': f"{year}-{month_str} Dasarian-1"
        },
        {
            'start': f"{year}-{month_str}-11T00:01",
            'end': f"{year}-{month_str}-20T23:59",
            'label': f"{year}-{month_str} Dasarian-2"
        },
        {
            'start': f"{year}-{month_str}-21T00:01",
            'end': f"{year}-{month_str}-{last_day}T23:59",
            'label': f"{year}-{month_str} Dasarian-3"
        }
    ]

    return periods

def process_dasarian_data(years=[2024], months=list(range(1, 13))):
    """Process dasarian data for given years and months"""
    dasarian_collection = []
    dasarian_labels = []
    export_tasks = []

    for year in years:
        for month in months:
            periods = create_dasarian_periods(year, month)

            for period in periods:
                print(f"Processing: {period['label']}")

                # Proses Sentinel-1 untuk periode dasarian
                processed_image = S1_preprocessing({
                    'start': period['start'],
                    'end': period['end']
                })

                # Hitung jumlah citra dalam periode dasarian
                image_count = processed_image.size()

                # Konversi band ke unsigned int16
                mosaic = processed_image.median()
                scaled_image = mosaic.select(["RPI", "VV_int", "VH_int", "API", "NDPI", "RVI"]) \
                                    .multiply(1000).toUint16() \
                                    .addBands(mosaic.select("angle").multiply(100).toUint16())

                final_image = scaled_image.clip(AOI).set({
                    'system:time_start': ee.Date(period['start']).millis(),
                    'system:time_end': ee.Date(period['end']).advance(1, 'day').millis(),
                    'dasarian': period['label'],
                    'year': year,
                    'month': month,
                    'image_count': image_count,
                    'scale_VV_int': 1000,
                    'scale_VH_int': 1000,
                    'scale_angle': 100
                })

                dasarian_collection.append(final_image)
                dasarian_labels.append(period['label'])

                # Prepare export task (tidak langsung dijalankan)
                export_task = {
                    'image': final_image,
                    'description': f'Sentinel1_{period["label"].replace(" ", "_")}',
                    'assetId': f'projects/try-spasial/assets/sentinel1_{period["label"].replace(" ", "_")}',
                    'scale': 30,
                    'region': AOI.geometry(),
                    'maxPixels': int(1e13)
                }

                export_tasks.append(export_task)

    # Gabungkan semua mosaic dasarian menjadi koleksi akhir
    final_dasarian_collection = ee.ImageCollection(dasarian_collection)

    return final_dasarian_collection, dasarian_labels, export_tasks

# ============================================================================
# JALANKAN PEMROSESAN
# ============================================================================

print("Memulai pemrosesan data Sentinel-1...")
final_collection, labels, tasks = process_dasarian_data([2024], list(range(1, 13)))

print(f"Berhasil membuat {len(labels)} periode dasarian")
print("Label periode yang tersedia:")
for i, label in enumerate(labels):
    print(f"{i+1}. {label}")

# ============================================================================
# VISUALISASI DATA
# ============================================================================

def visualize_dasarian(collection, labels, selected_index=0):
    """Visualize selected dasarian period"""
    if selected_index >= len(labels):
        print(f"Index {selected_index} tidak valid. Maksimal index: {len(labels)-1}")
        return

    selected_label = labels[selected_index]
    selected_image = collection.filter(ee.Filter.eq('dasarian', selected_label)).first()

    # Parameter visualisasi (disesuaikan dari kode asli)
    vis_params = {
        'bands': ['RPI', 'VV_int', 'VH_int'],
        'min': [43, 2, 165],  # Disesuaikan dari nilai asli yang dikali 1000
        'max': [273, 64, 813],  # Disesuaikan dari nilai asli yang dikali 1000
        'gamma': 1.0
    }
    Map.centerObject(AOI, 10)

    Map.addLayer(selected_image, vis_params, selected_label)
    print(f"Menampilkan: {selected_label}")

# visualisasi periode pertama
visualize_dasarian(final_collection, labels, 0)

# ============================================================================
# FUNGSI EXPORT (OPSIONAL)
# ============================================================================

def export_to_drive(export_tasks, start_index=0, end_index=None):
    """Export images to Google Drive"""
    if end_index is None:
        end_index = len(export_tasks)

    print(f"Memulai export dari index {start_index} sampai {end_index-1}...")

    for i in range(start_index, min(end_index, len(export_tasks))):
        task = export_tasks[i]

        export_task = ee.batch.Export.image.toDrive(
            image=task['image'],
            description=task['description'],
            folder='ARD_Sentinel-1',  # Nama folder di Google Drive
            fileNamePrefix=task['description'],  # Nama file di dalam folder
            scale=task['scale'],
            region=task['region'],
            maxPixels=task['maxPixels']
        )
        export_task.start()

        print(f"Export task dimulai: {task['description']}")
        print(f"Folder: ARD_Sentinel-1")
        print(f"Export ke Drive ID: {task['description']}")

export_to_drive(tasks)


# ============================================================================
# FUNGSI INTERAKTIF UNTUK MEMILIH PERIODE
# ============================================================================

def create_interactive_selector():
    """Create interactive selector for dasarian periods"""
    import ipywidgets as widgets
    from IPython.display import display

    # Dropdown widget untuk memilih periode
    period_dropdown = widgets.Dropdown(
        options=[(label, i) for i, label in enumerate(labels)],
        value=0,
        description='Periode:',
        style={'description_width': 'initial'}
    )

    def on_period_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            Map.clear_layers()
            Map.addLayer(AOI, {'color': 'red'}, 'AOI', False)
            visualize_dasarian(final_collection, labels, change['new'])

    period_dropdown.observe(on_period_change)

    display(period_dropdown)
    return period_dropdown

# Tampilam map
Map.addLayer(AOI, {'color': 'red'}, 'AOI', False)
print("Map siap ditampilkan!")
print("Gunakan Map untuk melihat hasil")

# Tampilkan map
Map

# Move assets to ImageCollection

In [ ]:
# Folder sumber (tempat asset berada)
source_folder = 'projects/try-spasial/assets/sentinel1'

# Folder tujuan (tempat asset akan dipindahkan)
destination_folder = 'projects/try-spasial/assets/collection'

print(f"Source folder: {source_folder}")
print(f"Destination folder: {destination_folder}")

def list_and_move_assets(source_folder, destination_folder):
    """List all assets in source folder and move them to ImageCollection with specified scale and dimensions"""
    try:
        # List all assets in the source folder
        asset_list = ee.data.listAssets({'parent': source_folder})

        if not asset_list.get('assets'):
            print("❌ No assets found in source folder!")
            return []

        print(f"Found {len(asset_list['assets'])} assets in the source folder:")
        print("=" * 50)

        # Membuat ImageCollection dari semua asset di folder
        images = []
        for asset in asset_list['assets']:
            asset_id = asset['id']
            image = ee.Image(asset_id)
            images.append(image)
            print(f"Asset: {asset_id}")

        # Gabungkan semua citra menjadi ImageCollection
        image_collection = ee.ImageCollection(images)

        # Menyimpan citra
        for image in images:
            asset_name = image.getInfo()['id'].split('/')[-1]  # Mengambil nama asset
            destination_path = f"{destination_folder}/{asset_name}"
            print(f"Moving asset to: {destination_path}")

            scale = 10  # Resolusi skala dalam meter
            region = image.geometry().bounds()  # Ambil batas geometri citra untuk menentukan region

            task = ee.batch.Export.image.toAsset(
                image=image,
                description=f"Export_{asset_name}",
                assetId=destination_path,
                scale=scale,
                region=region,
                maxPixels=int(1e13)
            )
            task.start()
            print(f"Export task started for {asset_name}")

        return image_collection

    except Exception as e:
        print(f"❌ Error processing assets: {str(e)}")
        return []

# Run the function to move files
list_and_move_assets(source_folder, destination_folder)